# **Installation**

In [ ]:
!pip install --upgrade --no-cache-dir "unsloth @ git+https://github.com/unslothai/unsloth.git@main"

  Cloning https://github.com/unslothai/unsloth.git (to revision main) to /tmp/pip-install-z0ko8pyd/unsloth_7ce17efb480743dd8cd8060a57c7f95e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-z0ko8pyd/unsloth_7ce17efb480743dd8cd8060a57c7f95e
  Resolved https://github.com/unslothai/unsloth.git to commit 6ac4e2e36f2f8bd0bc63a6eb85afa7097948ff3d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **Merge model fine-tuning and model base**

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "aismaanly/ai_comment",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.1: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.7.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

# **Data Prep**

## **Load CSV**

In [ ]:
from datasets import load_dataset
from google.colab import drive

drive.mount('/content/drive')

# Muat kembali dataset asli
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Dataset/dataset_posts.csv', sep=';')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

## **Split dataset**

In [ ]:
dataset_dict = dataset["train"].train_test_split(test_size=0.2)
eval_test_split = dataset_dict["test"].train_test_split(test_size=0.5)

dataset_dict["eval"] = eval_test_split["train"]
dataset_dict["test"] = eval_test_split["test"]

test_dataset = dataset_dict["test"]
print(test_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 339
})


# **Evaluation**

In [ ]:
from tqdm import tqdm
import pandas as pd
from transformers import pipeline, logging

logging.set_verbosity(logging.CRITICAL)

# Inisialisasi pipeline untuk generasi teks
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    return_full_text=False,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

In [ ]:
import re

predictions = []
references = [] # ori komentar
rows = []

print("Generating comments for evaluation...")

num_examples_to_evaluate = len(test_dataset)

for i in tqdm(range(num_examples_to_evaluate)):
    example = test_dataset[i]
    input_text = str(example.get("input", ""))
    original_comment = str(example.get("output", ""))

    formatted_prompt = data_prompt.format("", input_text, "")

    outputs = pipe(formatted_prompt)
    full_generated_text = outputs[0]["generated_text"]

    response_start_tag = "### Response:"
    if response_start_tag in full_generated_text:
        generated_comment = full_generated_text.split(response_start_tag, 1)[1].strip()
    else:
        generated_comment = full_generated_text.strip()

    generated_comment = generated_comment.replace(tokenizer.eos_token, "").strip()
    generated_comment = re.sub(r'<\|eot_id\|>', '', generated_comment).strip()

    if not generated_comment:
        generated_comment = "[EMPTY_GENERATED_COMMENT]"
    if not original_comment:
        original_comment = "[EMPTY_ORIGINAL_COMMENT]"

    predictions.append(generated_comment)
    references.append([original_comment])

    rows.append({
        "input_post": input_text,
        "original_comment": original_comment,
        "generated_comment": generated_comment,
    })

print("\nFinished generating comments.")

Generating comments for evaluation...


100%|██████████| 339/339 [04:32<00:00,  1.24it/s]


Finished generating comments.


In [ ]:
predictions_df = pd.DataFrame(rows)
predictions_df.sample(5)

,input_post,original_comment,generated_comment
15,ReminderHere’ what you should do when exfoliat...,"YESS, betul bgt mintific🙌💙",Keren banget tipsnya👏👏👏
208,Peeling for sensitive skin? Let’ find out!🎊Kam...,Mantul nih💙,Moist nya mantul😍
164,The truth about mixing NEW Sensitive Moisture ...,berarti cocok banget ya min dipakai setelah pa...,"Kulitku sensitif, apakah bisa pake ini?"
316,"Hari ini, Presidenresmi meluncurkan Daya Anaga...",.sahabat.odha dia emang oportunis,"kalo di urus oleh pemerintah, jadi seperti yg ..."
177,ReminderHere’ what you should do when exfoliat...,Ikan hiu melayang layanglove youusayangg 😘😘,"Min, panthenol serumnya boleh dipake setiap ha..."


In [ ]:
# Instalasi Perhitungan Metrik Evaluasi
!pip install evaluate
!pip install bert_score
!pip install numpy

In [ ]:
import evaluate
import numpy as np

# BERTScore
bertscore = evaluate.load("bertscore")
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

print("=== BERTScore ===")
print(f"BERTScore (Precision)    : {np.mean(bertscore_result['precision']):.4f}")
print(f"BERTScore (Recall)       : {np.mean(bertscore_result['recall']):.4f}")
print(f"BERTScore (F1)           : {np.mean(bertscore_result['f1']):.4f}")


=== BERTScore ===
BERTScore (Precision)    : 0.7900
BERTScore (Recall)       : 0.7833
BERTScore (F1)           : 0.7858
